In [1]:
import os
import re
import re
import json
import random
import pprint
import urllib3
import certifi
from time import sleep
from bs4 import BeautifulSoup

In [2]:
pp = pprint.PrettyPrinter(indent=2)

In [3]:
G = 'https://www.gutenberg.org'

In [80]:
author = 'Dickens' # 'Dickens, Charles'
authre = re.compile('^'+author, re.IGNORECASE)
lang = re.compile('English')
letter = author[0].lower()

In [81]:
url = 'http://www.gutenberg.org/browse/authors/' + letter

In [82]:
http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED', ca_certs=certifi.where())

In [83]:
r = http.request('GET', url)

In [84]:
soup = BeautifulSoup(r.data, 'html.parser')

In [85]:
h2 = soup.find_all(string=authre)
print(h2)

['Dickens, Charles, 1812-1870', "Dickens's Children: Ten Drawings", "Dickens' Stories About Children Every Child Can Read", 'Dickens, Mamie, 1838-1896', 'Dickens, Mary Angela, 1862-1948']


In [10]:
ul = h2[0].next
while ul.name != 'ul':
    ul = ul.next
print(ul)

<ul>
<li class="pgdbalias">Boz</li>
<li class="pgdbxlink"><a href="https://en.wikipedia.org/wiki/Charles_Dickens">en.wikipedia</a></li>
<li class="pgdbetext"><a href="/ebooks/18059">L'abîme</a> (French) (as Author)</li>
<li class="pgdbaudio"><a href="/ebooks/9691">All the Year Round</a> (English) (as Author)</li>
<li class="pgdbetext"><a href="/ebooks/675">American Notes</a> (English) (as Author)</li>
<li class="pgdbaudio"><a href="/ebooks/9693">American Notes</a> (English) (as Author)</li>
<li class="pgdbetext"><a href="/ebooks/13771">Aventures de Monsieur Pickwick, Vol. I</a> (French) (as Author)</li>
<li class="pgdbetext"><a href="/ebooks/14789">Aventures de Monsieur Pickwick, Vol. II</a> (French) (as Author)</li>
<li class="pgdbetext"><a href="/ebooks/37093">De avonturen van Oliver Twist</a> (Dutch) (as Author)</li>
<li class="pgdbetext"><a href="/ebooks/25985">Bardell v. Pickwick</a> (English) (as Author)</li>
<li class="pgdbetext"><a href="/ebooks/17879">Barnabé Rudge, Tome I</a>

In [11]:
as_auth = re.compile('as Author')
links = {}
for li in ul.find_all(class_='pgdbetext'):
    if li.find(string=as_auth) and li.find(string=lang):
        links[li.a.text.replace('\r',' ')] = li.a['href']

In [12]:
pp.pprint(links)

{ '"Old Scrooge": A Christmas Carol in Five Staves. Dramatized from Charles Dickens\' Celebrated Christmas Story.': '/ebooks/40729',
  "A Child's Dream of a Star": '/ebooks/42232',
  "A Child's History of England": '/ebooks/699',
  'A Christmas Carol': '/ebooks/24022',
  'A Christmas Carol in Prose; Being a Ghost Story of Christmas': '/ebooks/46',
  "A Christmas Carol; Or, The Miser's Warning! (Adapted from Charles Dickens' Celebrated Work.)": '/ebooks/41739',
  'A Christmas CarolThe original manuscript': '/ebooks/30368',
  'A House to Let': '/ebooks/2324',
  'A Message from the Sea': '/ebooks/1407',
  'A Tale of Two Cities': '/ebooks/98',
  'American Notes': '/ebooks/675',
  'Bardell v. Pickwick': '/ebooks/25985',
  "Barnaby Rudge: A Tale of the Riots of 'Eighty": '/ebooks/917',
  'Bleak House': '/ebooks/1023',
  'Captain Boldheart & the Latin-Grammar MasterA Holiday Romance from the Pen of Lieut-Col. Robin Redforth, aged 9': '/ebooks/23765',
  "Charles Dickens' Children Stories": '/e

In [13]:
test = list(links.items())[0]

In [163]:
r2 = http.request('GET', G+test[1])

In [186]:
soup2 = BeautifulSoup(r2.data, 'html.parser')

In [223]:
plainre = re.compile("plain text", re.IGNORECASE)

In [225]:
link = soup2(string=plainre)[0].parent['href']

In [226]:
r3 = http.request('GET', G+link)

In [236]:
print(r3.data[800:2500].decode('utf-8'))

f.org

                           [Picture: Emigrants]





                              AMERICAN NOTES
                                   FOR
                           GENERAL CIRCULATION
                                   AND
                         PICTURES FROM ITALY {1}


                                    BY
                             CHARLES DICKENS

                                * * * * *

                         WITH 8 ILLUSTRATIONS BY
                            MARCUS STONE, R.A.

                                * * * * *

                                  LONDON
                           CHAPMAN & HALL, LTD.
                                   1913

                                * * * * *

                           I DEDICATE THIS BOOK
                                    TO
                          THOSE FRIENDS OF MINE
                                IN AMERICA
                   WHO, GIVING ME A WELCOME I MUST EVER
                     GRATEFULLY AND PROUDLY 

In [23]:
author_dir = '-'.join(map(str.lower, author.split(', ')))
if not os.path.isdir(author_dir):
    os.mkdir(author_dir)
os.listdir()

['clean.py',
 'clean.ipynb',
 '.ipynb_checkpoints',
 'dickens-charles-clean',
 'dickens-charles',
 'scrape.ipynb',
 'scrape-author.ipynb']

In [14]:
rid = re.compile('[!?.()"";:,\']')

In [44]:
def get_ebook(link, secs=1):
    G = 'http://www.gutenberg.org'
    # go to list of formats
#     http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED', ca_certs=certifi.where())
    r = http.request('GET', G+link)
    s = BeautifulSoup(r.data, 'html.parser')
    # find utf-8 plain text version
    txt = re.compile('plain text', re.IGNORECASE)
    l = s(string=txt)[0].parent['href'] # link
    sleep(secs) # get thee not blocked
    # download ebook & return it
    h = http.request('Get', G+l)
    return(h.data)

In [45]:
def save_ebook(title, link):
    t = re.sub(rid, '', title.lower()).replace(' ', '-') + '.txt'
    if not os.path.isfile(os.path.join(author_dir, t)):
        ebook = get_ebook(link)
        print(f'saving {t}')
        with open(os.path.join(author_dir, t), 'wb') as f:
            f.write(ebook)
    else:
        print(f'already downloaded {t}, continuing')    

In [47]:
for title, link in links.items():
    save_ebook(title, link)

already downloaded american-notes.txt, continuing
saving bardell-v-pickwick.txt
saving barnaby-rudge-a-tale-of-the-riots-of-eighty.txt
saving the-battle-of-life.txt
saving the-battle-of-life-a-love-story.txt
saving bleak-house.txt
saving captain-boldheart-&-the-latin-grammar-mastera-holiday-romance-from-the-pen-of-lieut-col-robin-redforth-aged-9.txt
saving charles-dickens-children-stories.txt
saving a-childs-dream-of-a-star.txt
saving a-childs-history-of-england.txt
saving the-chimes-a-goblin-story-of-some-bells-that-rang-an-old-year-out-and-a-new-year-in.txt
saving a-christmas-carol.txt


KeyboardInterrupt: 